### Creating Model 1

* Creating a 1st model for the NFL Play data. The goal is to predict the play type on 4th down.
* Random Forest and gradient boosted trees are the other models

In [2]:
from pyspark.ml import Pipeline  
from pyspark.ml.feature import *  
from pyspark.ml.classification import RandomForestClassifier
import nfl_data_py as nfl
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTENC
import pandas as pd
from imblearn.combine import SMOTEENN
from DataPipelineFxn import *
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import functions as F

In [3]:
# get the spark session and data frame
data = GetSparkDF(include_undersample = True)

2000 done.
2001 done.
2002 done.
2003 done.
2004 done.
2005 done.
2006 done.
2007 done.
2008 done.
2009 done.
2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.


/opt/conda/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/12/04 01:05:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/04 01:05:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/04 01:05:50 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/12/04 01:05:51 WARN TaskSetManager: Stage 0 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.


24/12/04 01:05:53 WARN TaskSetManager: Stage 3 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.


24/12/04 01:05:59 WARN TaskSetManager: Stage 4 contains a task of very large size (2197 KiB). The maximum recommended task size is 1000 KiB.
24/12/04 01:06:03 WARN TaskSetManager: Stage 5 contains a task of very large size (1216 KiB). The maximum recommended task size is 1000 KiB.


In [4]:
spark, df, test_df, undersample_df = data

### Create numeric columns list for the pipeline

In [5]:
numeric_cols = []
for field in df.schema.fields:
    if str(field.dataType) != "StringType()":
        numeric_cols.append(field.name)
        
numeric_cols.remove("season")
numeric_cols.remove("week")

In [6]:
print(numeric_cols)

['yardline_100', 'quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining', 'down', 'drive', 'qtr', 'ydstogo', 'posteam_timeouts_remaining', 'defteam_timeouts_remaining', 'posteam_score', 'defteam_score', 'score_differential', 'ep', 'epa', 'wind', 'temp']


### Create the Model Pipeline

In [7]:

str_col = ["home_team", "away_team", "season_type", "posteam", "posteam_type", "defteam", "side_of_field", "game_half", "week",
        "play_type", "season", 'roof', 'surface']
str_col_output = ["home_team_idx", "away_team_idx", "season_type_idx", "posteam_idx", "posteam_type_idx", "defteam_idx", "week_idx",
                  "side_of_field_idx", "game_half_idx", "play_type_idx", "season_idx", 'roof_idx', 'surface_idx']
ohe_col_input = ["home_team_idx", "away_team_idx", "season_type_idx", "posteam_idx", "posteam_type_idx", "defteam_idx", "week_idx",
                  "side_of_field_idx", "game_half_idx", "season_idx", 'roof_idx', 'surface_idx']
ohe_col_vec = ["home_team_vec", "away_team_vec", "season_type_vec", "posteam_vec", "posteam_type_vec", "defteam_vec", "week_vec",
                  "side_of_field_vec", "game_half_ivec", "season_vec", 'roof_vec', 'surface_vec']

# Combine numeric and one-hot encoded columns
va_input_cols = numeric_cols + ohe_col_vec

# process using string indexer first for catgeorical features  
stringIndexer = StringIndexer(inputCols=str_col, outputCols=str_col_output)

# process rating data into second feature
ohe = OneHotEncoder(inputCols=ohe_col_input, outputCols=ohe_col_vec) 


# Assemble features column
va = VectorAssembler(inputCols=va_input_cols, outputCol="features", handleInvalid = "skip")

# process data using maxabs scaler, not necessarily important for trees but consistency
scaler = MaxAbsScaler(inputCol="features", outputCol="scaledFeatures")

# define the model

#spark.conf.set("spark.speculation", "false")
# Use Random Forest Classifier
rf = RandomForestClassifier(labelCol='play_type_idx', 
                             featuresCol='scaledFeatures', 
                             seed=42)


# Fit the pipeline
pipeline = Pipeline(stages=[stringIndexer, ohe, va, scaler, rf])

In [8]:

# Set up hyperparameter tuning
paramGrid = ParamGridBuilder()\
    .addGrid(rf.numTrees, [10, 50, 100])\
    .addGrid(rf.maxDepth, [5, 10, 15])\
    .addGrid(rf.impurity, ["gini", "entropy"])\
    .build()


# build the evaluator
evaluator = MulticlassClassificationEvaluator(
    predictionCol="prediction",
    labelCol="play_type_idx",
    metricName="accuracy"
)

# set up the train validation split
tvs = TrainValidationSplit(
    estimator=pipeline,
    evaluator=evaluator,
    estimatorParamMaps=paramGrid,
    # 80% of the data will be used for training, 20% for validation.
    trainRatio=0.8
)

In [ ]:
df.select(df.roof).distinct().show()

In [9]:
# need to replace the empty string in surface
df = df.withColumn(
    'surface',
    F.when(df.surface == '', 'Undefined').otherwise(df.surface)
)

# fit the model with the hyperparameter search
rf_model = tvs.fit(df)

24/12/04 01:06:52 WARN TaskSetManager: Stage 6 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.


24/12/04 01:06:54 WARN TaskSetManager: Stage 9 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.


24/12/04 01:06:56 WARN TaskSetManager: Stage 12 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.
24/12/04 01:06:56 WARN TaskSetManager: Stage 13 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.


24/12/04 01:06:57 WARN TaskSetManager: Stage 14 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.


24/12/04 01:06:59 WARN TaskSetManager: Stage 16 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.


...

24/12/04 01:11:23 WARN DAGScheduler: Broadcasting large task binary with size 1721.0 KiB


24/12/04 01:11:25 WARN DAGScheduler: Broadcasting large task binary with size 19.0 MiB
24/12/04 01:11:25 WARN TaskSetManager: Stage 618 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.


24/12/04 01:11:27 WARN DAGScheduler: Broadcasting large task binary with size 1980.3 KiB


24/12/04 01:11:30 WARN DAGScheduler: Broadcasting large task binary with size 24.2 MiB
24/12/04 01:11:30 WARN TaskSetManager: Stage 620 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.


24/12/04 01:11:33 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB


### Evaluate the Model

In [10]:
best_model = rf_model.bestModel

# Extract the parameter map of the best model
best_params = best_model.stages[-1].extractParamMap()

# Print the parameters values
print("Tuned Parameters of the Best Model:")
for param, value in best_params.items():
    print(f"{param.name}: {value}")

Tuned Parameters of the Best Model:
bootstrap: True
cacheNodeIds: False
checkpointInterval: 10
featureSubsetStrategy: auto
featuresCol: scaledFeatures
impurity: gini
labelCol: play_type_idx
leafCol: 
maxBins: 32
maxDepth: 15
maxMemoryInMB: 256
minInfoGain: 0.0
minInstancesPerNode: 1
minWeightFractionPerNode: 0.0
numTrees: 50
predictionCol: prediction
probabilityCol: probability
rawPredictionCol: rawPrediction
seed: 42
subsamplingRate: 1.0


In [11]:
# have to change test df instance
test_df = test_df.withColumn(
    'surface',
    F.when(test_df.surface == '', 'Undefined').otherwise(test_df.surface)
)

prediction = best_model.transform(test_df)


In [12]:
f1_evaluator = MulticlassClassificationEvaluator(
    labelCol='play_type_idx', predictionCol="prediction",
    metricName='f1'
)

accuracy_evaluator = MulticlassClassificationEvaluator(
    labelCol='play_type_idx', predictionCol="prediction",
    metricName='accuracy'
)

precision_by_label_eval = MulticlassClassificationEvaluator(
    labelCol='play_type_idx', predictionCol="prediction",
    metricName='precisionByLabel'
)

weighted_precision_eval = MulticlassClassificationEvaluator(
    labelCol='play_type_idx', predictionCol="prediction",
    metricName='weightedPrecision'
)


weighted_recall_eval = MulticlassClassificationEvaluator(
    labelCol='play_type_idx', predictionCol="prediction",
    metricName='weightedRecall'
)

f1_score = f1_evaluator.evaluate(prediction)
accuracy = accuracy_evaluator.evaluate(prediction)
precision_by_label = precision_by_label_eval.evaluate(prediction)
precision = weighted_precision_eval.evaluate(prediction)
recall = weighted_recall_eval.evaluate(prediction)

24/12/04 01:11:42 WARN TaskSetManager: Stage 622 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.
24/12/04 01:11:43 WARN TaskSetManager: Stage 623 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.


24/12/04 01:11:46 WARN DAGScheduler: Broadcasting large task binary with size 15.6 MiB


24/12/04 01:11:49 WARN TaskSetManager: Stage 632 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.
24/12/04 01:11:49 WARN TaskSetManager: Stage 633 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.


24/12/04 01:11:52 WARN DAGScheduler: Broadcasting large task binary with size 15.6 MiB


24/12/04 01:11:54 WARN TaskSetManager: Stage 642 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.
24/12/04 01:11:54 WARN TaskSetManager: Stage 643 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.


24/12/04 01:11:57 WARN DAGScheduler: Broadcasting large task binary with size 15.6 MiB


24/12/04 01:11:58 WARN TaskSetManager: Stage 652 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.
24/12/04 01:11:59 WARN TaskSetManager: Stage 653 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.


24/12/04 01:12:01 WARN DAGScheduler: Broadcasting large task binary with size 15.6 MiB


24/12/04 01:12:02 WARN TaskSetManager: Stage 662 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.
24/12/04 01:12:03 WARN TaskSetManager: Stage 663 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.


24/12/04 01:12:04 WARN DAGScheduler: Broadcasting large task binary with size 15.6 MiB


In [ ]:
prediction.select([prediction.play_type_idx, prediction.prediction]).show(10)

In [ ]:
prediction.groupby(prediction.prediction).count().show()

In [ ]:
prediction.groupby(prediction.play_type_idx).count().show()

In [13]:
labels = prediction.select([prediction.play_type, prediction.play_type_idx]).distinct().orderBy(prediction.play_type_idx).select(prediction.play_type).collect()
labels = [x[0] for x in labels]

24/12/04 01:12:10 WARN TaskSetManager: Stage 672 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.
24/12/04 01:12:10 WARN TaskSetManager: Stage 673 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.


In [ ]:
test_df.groupby(test_df.play_type).count().show()

In [14]:
results = f'''
F1: {f1_score}
Accuracy: {accuracy}
Precision by Label: {precision_by_label}
Weighted Precision: {precision}
Weighted Recall: {recall}
'''

print(results)


F1: 0.8791488975343991
Accuracy: 0.9015169628862145
Precision by Label: 0.939925224146067
Weighted Precision: 0.8651043570532826
Weighted Recall: 0.9015169628862145



In [15]:
from pyspark.sql.types import FloatType
from pyspark.mllib.evaluation import MulticlassMetrics

#Adapting code from https://www.kaggle.com/code/ashokkumarpalivela/multiclass-classification-using-pyspark for confusion matrix
preds_and_labels = prediction.select(['prediction','play_type_idx'])\
                              .withColumn('play_type_idx', col('play_type_idx')\
                              .cast(FloatType()))\
                              .orderBy('play_type_idx')
    
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

print(metrics.confusionMatrix().toArray())

24/12/04 01:12:22 WARN TaskSetManager: Stage 697 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.
24/12/04 01:12:23 WARN TaskSetManager: Stage 698 contains a task of very large size (6176 KiB). The maximum recommended task size is 1000 KiB.


24/12/04 01:12:25 WARN DAGScheduler: Broadcasting large task binary with size 15.6 MiB


24/12/04 01:12:28 WARN DAGScheduler: Broadcasting large task binary with size 15.6 MiB


/opt/conda/lib/python3.7/site-packages/pyspark/sql/context.py:159: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


24/12/04 01:12:30 WARN DAGScheduler: Broadcasting large task binary with size 15.5 MiB


24/12/04 01:12:31 WARN DAGScheduler: Broadcasting large task binary with size 15.5 MiB


[[2.5894e+04 2.3200e+02 5.4000e+01 7.0000e+00 0.0000e+00]
 [9.5000e+01 6.1030e+03 5.7000e+01 8.0000e+00 0.0000e+00]
 [3.7300e+02 5.2100e+02 1.3160e+03 1.6200e+02 0.0000e+00]
 [2.7200e+02 3.7800e+02 1.7700e+02 6.2100e+02 0.0000e+00]
 [9.1500e+02 2.7800e+02 1.2100e+02 5.7000e+01 0.0000e+00]]


In [20]:
_ = plt.figure(figsize=(10, 10))
ax = sns.heatmap(metrics.confusionMatrix().toArray(),
            cmap='viridis',
            annot=True,fmt='0',
            cbar=False, 
            xticklabels=labels,
            yticklabels=labels)

plt.title('Random Forest Regular Sampling Confusion Matrix', fontsize = 20) # title with fontsize 20
plt.xlabel('Predicted', fontsize = 15) # x-axis label with fontsize 15
plt.ylabel('Actual', fontsize = 15) # y-axis label with fontsize 15
plt.savefig('naive-bayes-normal-confusion-matrix.png')

TypeError: 'Figure' object is not callable

### Conclusions

Some quick observations:

* Much better performance when we don't undersample, could be because everything is a punt
* We need to be careful with the labels for the confusion matrix. They were not intially lining up properly with what the idxs are.
* I added temp, wind, roof, surface to the predictors